### Transactions

In [1]:
df1 = pd.read_csv('data/Azuki_BAYC_MAYC_Otherdeed_Moonbirds.csv')
df1 = df1[df1.__confirmed == True]

feats = [
    'timestamp',
    'exchange_name',
    'contract_version',
    
    'usd_price',
    'eth_price',
    'royalty_fee',
    'platform_fee',
    
    'collection_name',
    'token_id',
    
    'transaction_hash',
    'seller_address',
    'buyer_address',
    'is_multi_token_sale',
    'multi_token_sale_index',
]
df_trnx = df1[feats]

# features cleaning
df_trnx['timestamp'] = pd.to_datetime(df_trnx.timestamp)#.str[:10])
df_trnx['day']   = pd.to_datetime(df_trnx['timestamp'].astype(str).str[:10])
df_trnx['month'] = pd.to_datetime(df_trnx['timestamp'].astype(str).str[:7])
df_trnx['quarter'] = df_trnx['timestamp'].astype(str).str[:4] + 'Q' + pd.to_datetime(df_trnx['timestamp']).dt.quarter.astype(str)
df_trnx['year'] = df_trnx['timestamp'].astype(str).str[:4]
df_trnx['year_month'] = df_trnx.month.dt.year*100 + df_trnx.month.dt.month

feats0 = ['seller_address', 'buyer_address']
for f in feats0:
    df_trnx[f] = df_trnx[f].str.lower()

df_trnx['transaction_hash'] = df_trnx.transaction_hash.map(lambda x: f'<a href="https://etherscan.io/tx/{x}">url</a>')
df_trnx['trnx_len'] = df_trnx.groupby('transaction_hash')['token_id'].transform(len)

df_trnx.to_pickle('data/df_trnx.pkl')
print(df_trnx.shape)
df_trnx[:3]

(345518, 20)


,timestamp,exchange_name,contract_version,usd_price,eth_price,royalty_fee,platform_fee,collection_name,token_id,transaction_hash,seller_address,buyer_address,is_multi_token_sale,multi_token_sale_index,day,month,quarter,year,year_month,trnx_len
0,2023-03-23 11:15:23+00:00,opensea,seaport,"21,696.37",12.37,0.06,0.00,MutantApeYachtClub,10972,"<a href=""https://etherscan.io/tx/0x5f438f5b5dd...",0x0097b9cfe64455eed479292671a1121f502bc954,0xdf04f866d8e581982b442b613f8b77655425e545,False,0,2023-03-23,2023-03-01,2023Q1,2023,202303,1
1,2023-03-23 11:03:35+00:00,blur,v1,"21,129.12",12.06,0.06,0.00,MutantApeYachtClub,12974,"<a href=""https://etherscan.io/tx/0x92af5b4a0c8...",0x5e1416e0ec35b47a6b7687c5e783e9b98a83dd45,0x4c092f3597a5c81bcbc6746ed598c58a890d26ff,False,0,2023-03-23,2023-03-01,2023Q1,2023,202303,3
2,2023-03-23 11:03:35+00:00,blur,v1,"21,111.60",12.05,0.06,0.00,MutantApeYachtClub,15768,"<a href=""https://etherscan.io/tx/0x92af5b4a0c8...",0x5e1416e0ec35b47a6b7687c5e783e9b98a83dd45,0x7476a30f99109803dc289850412131dfba8e754d,False,0,2023-03-23,2023-03-01,2023Q1,2023,202303,3


In [2]:
# view transactions
f1 = df_trnx.collection_name == 'BoredApeYachtClub'
f2 = df_trnx.token_id == 9991

display(HTML(df_trnx[f1&f2].sort_values('timestamp')[:].to_html(render_links=True, escape=False)))

,timestamp,exchange_name,contract_version,usd_price,eth_price,royalty_fee,platform_fee,collection_name,token_id,transaction_hash,seller_address,buyer_address,is_multi_token_sale,multi_token_sale_index,day,month,quarter,year,year_month,trnx_len
245482,2022-01-24 07:03:06+00:00,opensea,wyvern,"1,864.45",0.77,0.02,0.02,BoredApeYachtClub,9991,url,0x7949eb8e7b8d83eaa26d8ad2854e6d3c59da6813,0xb1a22cc48f6784f629a994917cd6474923630c48,False,0,2022-01-24,2022-01-01,2022Q1,2022,202201,1
245479,2022-01-24 07:25:03+00:00,opensea,wyvern,"204,113.43",84.20,2.10,2.10,BoredApeYachtClub,9991,url,0xb1a22cc48f6784f629a994917cd6474923630c48,0x33d4b1c41ba877e97cb366e4c53e426cc898896b,False,0,2022-01-24,2022-01-01,2022Q1,2022,202201,1
244689,2022-02-27 04:46:31+00:00,opensea,wyvern,"338,108.39",124.50,3.11,3.11,BoredApeYachtClub,9991,url,0x33d4b1c41ba877e97cb366e4c53e426cc898896b,0xa46e9aaa2f6955af427dad3b5ae524e464db842b,False,0,2022-02-27,2022-02-01,2022Q1,2022,202202,1
239030,2022-12-27 19:06:59+00:00,opensea,seaport,"91,753.28",76.00,1.90,1.90,BoredApeYachtClub,9991,url,0xa46e9aaa2f6955af427dad3b5ae524e464db842b,0xed2ab4948ba6a909a7751dec4f34f303eb8c7236,False,0,2022-12-27,2022-12-01,2022Q4,2022,202212,5
238529,2023-01-14 17:17:47+00:00,blur,v1,"109,933.45",71.76,0.36,0.00,BoredApeYachtClub,9991,url,0xed2ab4948ba6a909a7751dec4f34f303eb8c7236,0x35a0af9b7d25671eaa27315eecd5c9087ba2944d,False,0,2023-01-14,2023-01-01,2023Q1,2023,202301,16
238435,2023-01-18 23:21:35+00:00,blur,v1,"107,718.04",70.61,0.35,0.00,BoredApeYachtClub,9991,url,0x35a0af9b7d25671eaa27315eecd5c9087ba2944d,0x1c66f8a8fa9d34d26b6767cca81e4f0fb8f0692f,False,0,2023-01-18,2023-01-01,2023Q1,2023,202301,1


### NFT metadata

In [3]:
# collection_name + token_id - unique NFT identifier
feats = [
    'collection_name', 
    'token_id',
    'image_url',
    'flattened_properties',
    'minted_timestamp',    
    'last_refreshed', 
]
# cleaning
df_nft = df1.groupby(feats)['eth_price'].agg([len,sum]).reset_index().rename(columns = {'len':'trnx_cnt','sum':'trnx_sum'})

df_nft['minted_timestamp'] = pd.to_datetime(df_nft.minted_timestamp.astype(str).str[:10])
df_nft['last_refreshed'] = pd.to_datetime(df_nft.last_refreshed.astype(str).str[:10])

df_nft['flattened_properties'] = df_nft.flattened_properties.map(eval)

df_nft['image_url'] = df_nft.image_url.map(lambda x: f'<img style="max-height:80px;" src="{x}">')

df_nft.to_pickle('data/df_nft.pkl')
print(df_nft.shape)
df_nft[:3]

(83470, 8)


,collection_name,token_id,image_url,flattened_properties,minted_timestamp,last_refreshed,trnx_cnt,trnx_sum
0,Azuki,44,"<img style=""max-height:80px;"" src=""https://ikz...","{'Eyes': 'Determined', 'Hair': 'Blonde Flowy',...",2022-01-12,2023-02-03,9,127.25
1,Azuki,53,"<img style=""max-height:80px;"" src=""https://ikz...","{'Eyes': 'Determined', 'Face': 'Blue Sunglasse...",2022-01-12,2023-02-02,3,34.06
2,Azuki,58,"<img style=""max-height:80px;"" src=""https://ikz...","{'Eyes': 'Tired', 'Hair': 'Green Fluffy', 'Typ...",2022-01-12,2023-02-02,3,44.89


In [4]:
# collect the dictionary of flattened NFT collection tables
dict_nft = dict()

for c in df_nft.collection_name.unique():
    df3 = df_nft[df_nft.collection_name == c].reset_index()
    del df3['index']
    df4 = pd.DataFrame(list(df3.flattened_properties))
    df4 = pd.concat([df3, df4], axis = 1)
    del df4['flattened_properties']
    df4 = df4.sort_values('trnx_cnt', ascending = False)
    dict_nft[c] = df4
      
    print('------------')
    print(df4.shape)
    # print(df4.isnull().sum())
    display(HTML(df4[:2].to_html(render_links=True, escape=False)))

------------
(9166, 19)


,collection_name,token_id,image_url,minted_timestamp,last_refreshed,trnx_cnt,trnx_sum,Eyes,Hair,Neck,Type,Mouth,Offhand,Clothing,Background,Face,Headgear,Ear,Special
6580,Azuki,7149,,2022-01-12,2023-02-05,106,"1,360.82",Relaxed,Orange Pixie,NaN,Human,Smile,NaN,Light Kimono,Off White B,NaN,NaN,NaN,NaN
6191,Azuki,6741,,2022-01-12,2023-02-06,99,"1,255.60",Closed,Silver Pixie,NaN,Human,Wheat Straw,Katana,Blue Floral Kimono,Off White A,Sleep Mask,NaN,NaN,NaN


------------
(8915, 14)


,collection_name,token_id,image_url,minted_timestamp,last_refreshed,trnx_cnt,trnx_sum,Fur,Hat,Eyes,Mouth,Clothes,Background,Earring
1684,BoredApeYachtClub,2045,,2021-05-01,2023-01-28,60,"3,723.19",Cream,Irish Boho,X Eyes,Dumbfounded,Hawaiian,Blue,NaN
1816,BoredApeYachtClub,2193,,2021-05-01,2023-01-28,59,"3,879.18",Cream,Girl's Hair Short,Coins,Bored Unshaven,Sleeveless T,Purple,NaN


------------
(6673, 15)


,collection_name,token_id,image_url,minted_timestamp,last_refreshed,trnx_cnt,trnx_sum,Beak,Body,Eyes,Eyewear,Feathers,Background,Headwear,Outerwear
5718,Moonbirds,8424,,2022-04-16,2023-02-06,116,830.26,Small,Professor,Side-eye,NaN,Gray,Green,NaN,NaN
6635,Moonbirds,9836,,2022-04-16,2023-02-06,101,744.53,Small,Professor,Angry,NaN,Purple,Yellow,NaN,NaN


------------
(13905, 15)


,collection_name,token_id,image_url,minted_timestamp,last_refreshed,trnx_cnt,trnx_sum,Fur,Hat,Eyes,Mouth,Clothes,Background,Earring,Name
8128,MutantApeYachtClub,9199,,2021-08-29,2023-02-04,86,"1,303.14",M1 Cream,NaN,M1 Bored,M1 Phoneme Wah,M1 Wool Turtleneck,M1 Army Green,NaN,NaN
12412,MutantApeYachtClub,24056,,2021-08-29,2023-02-06,82,"1,181.22",M1 Dark Brown,M1 Police Motorcycle Helmet,M1 Eyepatch,M1 Bored Unshaven,NaN,M1 Gray,NaN,NaN


------------
(44811, 24)


,collection_name,token_id,image_url,minted_timestamp,last_refreshed,trnx_cnt,trnx_sum,Plot,Artifact,Category,Sediment,Environment,Sediment Tier,Eastern Resource,Environment Tier,Western Resource,Eastern Resource Tier,Western Resource Tier,Northern Resource,Northern Resource Tier,Southern Resource,Southern Resource Tier,Obelisk Piece,Koda
42265,Otherdeed,95769,,2022-05-01,2023-02-02,103,159.39,95769,NaN,Harsh,Cosmic Dream,Glacia,3,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40627,Otherdeed,93056,,2022-05-01,2023-02-02,100,"2,712.96",93056,NaN,Mineral,Cosmic Dream,Biolume,1,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
